# LFP example 


In [1]:

from bayes_window.generative_models import generate_fake_lfp
from bayes_window.visualization import plot_data, plot_data_slope_trials
from bayes_window import visualization
from importlib import reload


## Model oscillation power
40 trials of "theta power" is generated for every animal. It is drawn randomly as a poisson process. 

This is repeated for "stimulation" trials, but poisson rate is higher.

Mice vary in their baseline power. Higher-baseline mice tend to have smaller stim response (See plot)

In [2]:

df, df_monster, index_cols, _ = generate_fake_lfp(mouse_response_slope=13,
                                                 n_trials=40)

In [3]:
reload(visualization)
c1=visualization.plot_data(df=df,x='stim',y='Log power').properties(width=60)
c2=visualization.plot_data_slope_trials(df=df,x='stim',y='Log power',color=None,detail='i_trial')
(c1+c2).facet(column='mouse')

alt.FacetChart(...)

In [4]:
reload(visualization)
visualization.plot_data(df=df,x='stim',y='Log power',color='mouse').properties(width=80)

alt.LayerChart(...)

In [5]:
c1.facet('mouse')

alt.FacetChart(...)

In [6]:
reload(visualization)
visualization.plot_data_slope_trials(df=df,x='stim',y='Power',detail='i_trial').properties().facet(column='mouse')


alt.FacetChart(...)

## Fit and plot slopes

In [25]:
from bayes_window import workflow
from bayes_window import models
from bayes_window import visualization
from importlib import reload
reload(workflow)
reload(visualization)
reload(models)
bw=workflow.BayesWindow(df,y='Log power', treatment='stim', group='mouse')
bw.fit_slopes(add_data=True, model=models.model_hier_stim_one_codition,
              do_make_change='subtract', dist_y='normal')

n(Divergences) = 0


In [26]:
intercepts=bw.trace.a_subject.mean(['chain','draw']).to_dataframe().reset_index()
import altair as alt 
chart1=alt.Chart(intercepts).mark_bar().encode(
    x=alt.X('subject:O',title='Mouse'),
    y=alt.Y('a_subject', title='Intercept')
)
chart1

alt.Chart(...)

In [27]:
bw.plot_posteriors_slopes(add_box=True, independent_axes=False)

alt.LayerChart(...)

In [30]:
chart1|bw.plot_posteriors_slopes(add_box=True, independent_axes=False)

alt.HConcatChart(...)

In [29]:
chart1|bw.plot_posteriors_slopes(add_box=True, independent_axes=True)

alt.HConcatChart(...)

In [23]:
bw.chart


alt.LayerChart(...)